In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}

In [4]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_7140\2020992259.py:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,37,41,43,47,51,75,76,77,78,79,80,81,82,83,84,85,86,87,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)


In [41]:
COLUMNAS_SERVICIO_TRASLADO = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}

COLUMNAS_FECHA_TRASLADO = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}


# Filtra la base nacional para dejar el Torax
df_torax = df_procesada.query("COD_HOSPITAL == 112103")
# Permite identificar cada egreso
df_torax = df_torax.reset_index().reset_index(names="id_egreso")

In [48]:
# Convierte los servicios y las fechas en formato largo
servicios_egreso_long = df_torax.melt(
    id_vars="id_egreso",
    value_vars=COLUMNAS_SERVICIO_TRASLADO,
    var_name="tipo_servicio",
    value_name="servicio",
)

fechas_egreso_long = df_torax.melt(
    id_vars="id_egreso",
    value_vars=COLUMNAS_FECHA_TRASLADO,
    var_name="tipo_fecha",
    value_name="fecha",
)

In [90]:
# Une los servicios y fechas respectivas para que esten en formato contiguo
servicio_y_fechas_egreso_contiguas = pd.merge(
    left=fechas_egreso_long,
    right=servicios_egreso_long,
    how="inner",
    left_index=True,
    right_index=True,
)

# Indica el tipo de ordenamiento del tipo de servicio ("SERVICIONGR", "SERVICIO1", "SERVICIOALTA")
servicio_y_fechas_egreso_contiguas["tipo_servicio"] = pd.Categorical(
    servicio_y_fechas_egreso_contiguas["tipo_servicio"],
    categories=COLUMNAS_SERVICIO_TRASLADO.keys(),
    ordered=True,
)

# Ordena cada egreso segun su tipo de servicio ("SERVICIONGR", "SERVICIO1", "SERVICIOALTA")
servicio_y_fechas_egreso_contiguas = servicio_y_fechas_egreso_contiguas.sort_values(
    ["id_egreso_x", "tipo_servicio"]
)

# Elimina filas sin ninguna fecha ingresada
servicio_y_fechas_egreso_contiguas = servicio_y_fechas_egreso_contiguas.dropna()

# Imputa fechas con formato de fechas
servicio_y_fechas_egreso_contiguas["fecha"] = pd.to_datetime(
    servicio_y_fechas_egreso_contiguas["fecha"]
)

In [91]:
servicio_y_fechas_egreso_contiguas["tiempo_de_estancia"] = (
    servicio_y_fechas_egreso_contiguas.groupby("id_egreso_x")["fecha"].diff().shift(-1)
)

In [92]:
servicio_y_fechas_egreso_contiguas.head(50)

,id_egreso_x,tipo_fecha,fecha,id_egreso_y,tipo_servicio,servicio,tiempo_de_estancia
0,0,FECHA_INGRESO,2019-01-14,0,SERVICIOINGRESO,CIRUGÍA CARDIOVASCULAR,3 days
134660,0,FECHAALTA,2019-01-17,0,SERVICIOALTA,CIRUGÍA CARDIOVASCULAR,NaT
1,1,FECHA_INGRESO,2019-08-26,1,SERVICIOINGRESO,AREA MÉDICA,0 days
134661,1,FECHAALTA,2019-08-26,1,SERVICIOALTA,AREA MÉDICA,NaT
2,2,FECHA_INGRESO,2019-01-09,2,SERVICIOINGRESO,AREA MÉDICA,0 days
134662,2,FECHAALTA,2019-01-09,2,SERVICIOALTA,AREA MÉDICA,NaT
3,3,FECHA_INGRESO,2019-02-18,3,SERVICIOINGRESO,UNIDAD DE RECUPERACIÓN DE PABELLONES (CENTRAL ...,0 days
134663,3,FECHAALTA,2019-02-18,3,SERVICIOALTA,UNIDAD DE RECUPERACIÓN DE PABELLONES (CENTRAL ...,NaT
4,4,FECHA_INGRESO,2019-06-18,4,SERVICIOINGRESO,UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR,11 days
13470,4,FECHATRASLADO1,2019-06-29,4,SERVICIOTRASLADO1,UNIDAD DE CUIDADOS INTENSIVOS ADULTO,1 days
